### 크롤러 준비 (다른 크롤러들은 url 첨부파일을 긁어오는 양식이 혼재되어 있어서 일단 이쪽에 맞춰 둠)

In [1]:
# K-Startup 정부지원사업 첨부파일 크롤러

from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd

import os
import time
import requests

from tqdm import trange
from urllib import request
from urllib.request import urlretrieve


crawlingfile = pd.DataFrame(columns=['url', 'file_name'])

site = "https://www.k-startup.go.kr/"
mydir ='C:/Users/smhrd/Desktop/지원사업/K-Startup'

if not os.path.isdir('C:/Users/smhrd/Desktop/지원사업') :
    # 폴더를 생성하는 코드작성
    os.mkdir('C:/Users/smhrd/Desktop/지원사업')
else :
    print("이미 폴더가 존재합니다")
    
driver = wd.Chrome()
driver.get('https://www.k-startup.go.kr/web/contents/bizpbanc-ongoing.do#none')

soup = bs(driver.page_source,'lxml')
page_count = soup.select('#bizPbancList > div > a')

try :
    for i in range(len(page_count)) :
        page_click = driver.find_element(By.CSS_SELECTOR, f'#bizPbancList > div > a:nth-child({i+1})')
        page_click.click()
        time.sleep(1)

        soup = bs(driver.page_source,'lxml')
        title_count = soup.select('.middle .tit')

        for i in range(len(title_count)) :
            title_click = driver.find_element(By.CSS_SELECTOR, f'#bizPbancList > ul > li:nth-child({i+1}) > div > div.right > div.middle > a')
            title_click.click()
            site_url = driver.current_url

            soup = bs(driver.page_source,'lxml')

            file = soup.select('.btn_down')
            file_type = soup.select('.file_bg')

            file_list = []
            for i in range(len(file)) :
                file_list.append(site+file[i]['href'])

            type_list = []
            for i in range(len(file_type)) :
                type_list.append(mydir+file_type[i].text)
                # 사이트 url과 파일명을 미리 저장하겠음
                crawlingfile.loc[len(crawlingfile)] = [site_url, 'K-Startup'+file_type[i].text]

            for i in range(len(file_list)) :
                urlretrieve(file_list[i],type_list[i])
                

            # time.sleep(5)
            driver.back()
except Exception as e :
    print(e)

In [2]:
# 파일 url을 가져올 수 없어서 저장하여 src로 처리 / + 사이트 url과 파일명 리스트
# 첨부파일 없는 사이트들이 혼재되어 있음
crawlingfile

,url,file_name
0,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup붙임1.공고문 hwp.hwp
1,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup붙임2. 포스터.png
2,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup붙임3. 썸썸스페이스 창업 프로젝트 참가신청서(초기창업).hwp
3,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup공모전_소공인창업오디션_포스터.png
4,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp
...,...,...
173,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup[공고문] 2023년 창업중심대학 모집공고.hwp
174,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf
175,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup[붙임1] 2023년 창업중심대학 사업계획서 양식.hwp
176,https://www.k-startup.go.kr/web/contents/bizpb...,K-Startup[붙임2] 2023년 창업중심대학 사업계획서 증빙자료 양식.hwp


In [6]:
# 첨부파일 없는 사이트 제거
mask = crawlingfile['file_name'].isin([[]])
craw_file = crawlingfile[~mask]
craw_file

,url,file_name
0,https://www.k-startup.go.kr/web/contents/bizpb...,붙임1.공고문 hwp.hwp
1,https://www.k-startup.go.kr/web/contents/bizpb...,붙임2. 포스터.png
2,https://www.k-startup.go.kr/web/contents/bizpb...,붙임3. 썸썸스페이스 창업 프로젝트 참가신청서(초기창업).hwp
3,https://www.k-startup.go.kr/web/contents/bizpb...,공모전_소공인창업오디션_포스터.png
4,https://www.k-startup.go.kr/web/contents/bizpb...,1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp
...,...,...
87,https://www.k-startup.go.kr/web/contents/bizpb...,IPO 기술특례상장 지원사업 설명회 포스터.jpg
88,https://www.k-startup.go.kr/web/contents/bizpb...,2022년 INU 스타트업 아카데미 포스터.jpg
89,https://www.k-startup.go.kr/web/contents/bizpb...,붙임 1. 2022 INU 스타트업 아카데미 공고문.hwp
90,https://www.k-startup.go.kr/web/contents/bizpb...,붙임 2. 신청서 및 제출서류 양식.hwp


In [7]:
# 인덱스가 꼬였으므로 인덱스를 초기화한다.
craw_file.reset_index(inplace=True)

In [8]:
craw_file

,index,url,file_name
0,0,https://www.k-startup.go.kr/web/contents/bizpb...,붙임1.공고문 hwp.hwp
1,1,https://www.k-startup.go.kr/web/contents/bizpb...,붙임2. 포스터.png
2,2,https://www.k-startup.go.kr/web/contents/bizpb...,붙임3. 썸썸스페이스 창업 프로젝트 참가신청서(초기창업).hwp
3,3,https://www.k-startup.go.kr/web/contents/bizpb...,공모전_소공인창업오디션_포스터.png
4,4,https://www.k-startup.go.kr/web/contents/bizpb...,1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp
...,...,...,...
87,87,https://www.k-startup.go.kr/web/contents/bizpb...,IPO 기술특례상장 지원사업 설명회 포스터.jpg
88,88,https://www.k-startup.go.kr/web/contents/bizpb...,2022년 INU 스타트업 아카데미 포스터.jpg
89,89,https://www.k-startup.go.kr/web/contents/bizpb...,붙임 1. 2022 INU 스타트업 아카데미 공고문.hwp
90,90,https://www.k-startup.go.kr/web/contents/bizpb...,붙임 2. 신청서 및 제출서류 양식.hwp


### 크롤러로 가져온 파일종류에 따라 분류

In [3]:
import os

hwp_file = []
pdf_file = []
img_file = [] 

dir_path = "C:/Users/smhrd/Desktop/지원사업"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_type = file.split('.')[-1]
        if file_type == 'hwp' :
            hwp_file.append(file)
        elif file_type == 'pdf' :
            pdf_file.append(file)
        else :
            img_file.append(file)
            
print('hwp : ', hwp_file)
print('pdf : ', pdf_file)
print('img : ', img_file)

hwp :  ['K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp', 'K-Startup(붙임1) 2023년 생애최초 청년창업 지원사업 주관기관 사업계획서 양식.hwp', 'K-Startup(붙임2) 2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자료 양식.hwp', 'K-Startup(붙임2) 2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료 양식.hwp', 'K-Startup01-2023년 산업맞춤형 혁신 바우처 지원사업 공급기업 모집 공고.hwp', 'K-Startup02-2023년 공급기업 Pool 등록 신청양식.hwp', 'K-Startup1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp', 'K-Startup2. 제1회 IPO 기술특례상장 지원사업 설명회 기술사업계획서 양식.hwp', 'K-Startup2023년 공공기술 창업사업화 지원사업 주관기관 모집공고.hwp', 'K-Startup2023년 생애최초 청년창업 지원사업 주관기관 모집공고.hwp', 'K-Startup3. 제1회 IPO 기술특례상장 지원사업 설명회 참가 신청서.hwp', 'K-Startup4. 제1회 IPO 기술특례상장 지원사업 설명회 개인정보동의서.hwp', 'K-Startup[공고문] 2023년 창업중심대학 모집공고.hwp', 'K-Startup[붙임1] 2023년 창업중심대학 사업계획서 양식.hwp', 'K-Startup[붙임2] 2023년 창업중심대학 사업계획서 증빙자료 양식.hwp', 'K-Startup★ [붙임 1] 모집안내문_REUP 프로젝트 3차 (추가).hwp', 'K-Startup붙임 1. 2022 INU 스타트업 아카데미 공고문.hwp', 'K-Startup붙임 2. 신청서 및 제출서류 양식.hwp', 'K-Startup붙임1.공고문 hwp.hwp', 'K-Startup붙임3. 썸썸스페이스 창업 프로젝트 참가신청서(초기창업).hwp', 'K-Startup창업기업 모집공고문.hwp', 'K-S

In [5]:
print(len(hwp_file))
print(len(pdf_file))
print(len(img_file))

22
5
14


#### 3-1) 한글파일일 경우

In [6]:
!pip install olefile
!pip install --user --pre pyhwp

In [7]:
# 한글파일 텍스트화 및 저장

import olefile

hwp_texts = []

dir_path # 경로
hwp_file # 한글파일이름 리스트

for i in range(len(hwp_file)) :
    f = olefile.OleFileIO(dir_path+'/'+hwp_file[i])
    encoded_text = f.openstream('PrvText').read()
    decoded_text = encoded_text.decode('UTF-16')
    hwp_texts.append(decoded_text)

hwp_texts

['<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<▪ 온라인 신청 방법   ① 신청방법 : K-Startup 누리집 (www.k-startup.go.kr) 로그인 → 사업신청관리 클릭 → ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고’ 클릭 후 작성 및 신청    * 세부 신청방법은 ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고문’ 참고   ② 제출서류 : 가. ‘제출 공문’(신청기관 대표자 명의) → hwp, pdf 형식만 가능나. ‘사업계획서’(A4 크기) → hwp 형식만 가능(필수)다. ‘증빙서류’(A4 크기) → hwp, pdf 형식만 가능     * 파일명 : (기관명)_공공기술 창업사업화 지원사업 사업계획서.hwp (파일명 임의변경불가)    * 파일제한용량은 300MB (제출서류 전체)이며, 타 형식의 문서는 접수 불가    * 서류별 제출 형식(hwp, pdf)을 참고하여 시스템 업로드   ③ 접수확인 및 완료  ▪ 온라인 신청 기간 : 2022. 12. 22.(목) ~ 2023. 1. 11.(수), 16:00     * 제출기한 내 온라인 신청을 모두 완료하여야 함(반드시 기한 내 시간 엄수)>\r\n<2. 유의사항>\r\n<※ ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고문’과 사업계획서 양식 내 ’작성방법‘ 등 내용을 숙지한 후 작성하여야 하며, 관련 내용의 미숙지나 기관 귀책 사유로 인하여 발생하는 불이익에 대한 책임은 신청기관에 있음  ※ 사업계획서에 작성된 모든 내용은 허위사실이 없어야 하며, 명시된 사항이 허위로 밝혀질 경우 감점 또는 선정 취소 등의 제재조치를 할 수 있음  ※ 사업 신청기간 내 사업계획서 제출을 필히 완료해야 하며, 마감 이후 추가 제출 불가    ▪ 국문 작성이 원칙이며 한글 2010 이상 버전으로 작성   ▪ 장평 100%, 줄 간격 160%, 휴먼명조체 (필요 시 일부조정 가능)   ▪ 용지 : 편집용지(위쪽 10, 아래쪽 10,

In [8]:
# 잘 들어갔는지 확인
hwp_texts[0]

'<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<▪ 온라인 신청 방법   ① 신청방법 : K-Startup 누리집 (www.k-startup.go.kr) 로그인 → 사업신청관리 클릭 → ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고’ 클릭 후 작성 및 신청    * 세부 신청방법은 ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고문’ 참고   ② 제출서류 : 가. ‘제출 공문’(신청기관 대표자 명의) → hwp, pdf 형식만 가능나. ‘사업계획서’(A4 크기) → hwp 형식만 가능(필수)다. ‘증빙서류’(A4 크기) → hwp, pdf 형식만 가능     * 파일명 : (기관명)_공공기술 창업사업화 지원사업 사업계획서.hwp (파일명 임의변경불가)    * 파일제한용량은 300MB (제출서류 전체)이며, 타 형식의 문서는 접수 불가    * 서류별 제출 형식(hwp, pdf)을 참고하여 시스템 업로드   ③ 접수확인 및 완료  ▪ 온라인 신청 기간 : 2022. 12. 22.(목) ~ 2023. 1. 11.(수), 16:00     * 제출기한 내 온라인 신청을 모두 완료하여야 함(반드시 기한 내 시간 엄수)>\r\n<2. 유의사항>\r\n<※ ‘2023년 공공기술 창업사업화 지원사업 주관기관 모집 공고문’과 사업계획서 양식 내 ’작성방법‘ 등 내용을 숙지한 후 작성하여야 하며, 관련 내용의 미숙지나 기관 귀책 사유로 인하여 발생하는 불이익에 대한 책임은 신청기관에 있음  ※ 사업계획서에 작성된 모든 내용은 허위사실이 없어야 하며, 명시된 사항이 허위로 밝혀질 경우 감점 또는 선정 취소 등의 제재조치를 할 수 있음  ※ 사업 신청기간 내 사업계획서 제출을 필히 완료해야 하며, 마감 이후 추가 제출 불가    ▪ 국문 작성이 원칙이며 한글 2010 이상 버전으로 작성   ▪ 장평 100%, 줄 간격 160%, 휴먼명조체 (필요 시 일부조정 가능)   ▪ 용지 : 편집용지(위쪽 10, 아래쪽 10, 

In [9]:
hwp_files = pd.DataFrame(columns=['file_name', 'contents'])

for i in range (len(hwp_file)) :
    hwp_files.loc[len(hwp_files)] = [hwp_file[i], hwp_texts[i]]

hwp_files

,file_name,contents
0,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...
1,K-Startup(붙임1) 2023년 생애최초 청년창업 지원사업 주관기관 사업계획서...,<1. 작성요령(신청 및 제출 방법)>< ※삭제 후 제출>\r\n<▪ 온라인 신청 ...
2,K-Startup(붙임2) 2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자...,\r\n\r\n<2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자료>\r\n...
3,K-Startup(붙임2) 2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료...,\r\n\r\n<2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료>\r\n\...
4,K-Startup01-2023년 산업맞춤형 혁신 바우처 지원사업 공급기업 모집 공고...,<>\r\n<2023년 산업맞춤형 혁신 바우처 지원사업 공급기업(관) 모집공고>\r...
5,K-Startup02-2023년 공급기업 Pool 등록 신청양식.hwp,<첨부1><><2023년 산업 맞춤형 혁신 바우처 공급기업(관) Pool 등록 신청...
6,K-Startup1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp,<제1회 IPO 기술특례상장 지원사업 설명회>\r\n\r\n기술력과 성장...
7,K-Startup2. 제1회 IPO 기술특례상장 지원사업 설명회 기술사업계획서 양식...,<<><><><><><><><><>\r\n<><><><><><><><><>\r\n<...
8,K-Startup2023년 공공기술 창업사업화 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 596호\r\n\r\n’23년 공공기술 창업사업화...
9,K-Startup2023년 생애최초 청년창업 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 595호\r\n\r\n’23년 생애최초 청년창업 ...


In [11]:
df_inner = pd.merge(hwp_files,crawlingfile,left_on = 'file_name',right_on='file_name',how='inner')
df_inner

,file_name,contents,url
0,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
1,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
2,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
3,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
4,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
...,...,...,...
108,K-Startup창업기업 사업계획서.hwp,<「2023 BIGS 부산 창업기업 지원사업」 사 업 계 획 서>\r\n * 본문 ...,https://www.k-startup.go.kr/web/contents/bizpb...
109,K-Startup창업기업 사업계획서.hwp,<「2023 BIGS 부산 창업기업 지원사업」 사 업 계 획 서>\r\n * 본문 ...,https://www.k-startup.go.kr/web/contents/bizpb...
110,K-Startup창업기업 사업계획서.hwp,<「2023 BIGS 부산 창업기업 지원사업」 사 업 계 획 서>\r\n * 본문 ...,https://www.k-startup.go.kr/web/contents/bizpb...
111,K-Startup창업기업 사업계획서.hwp,<「2023 BIGS 부산 창업기업 지원사업」 사 업 계 획 서>\r\n * 본문 ...,https://www.k-startup.go.kr/web/contents/bizpb...


In [12]:
hwp_drop = df_inner.drop_duplicates()
hwp_drop

,file_name,contents,url
0,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
6,K-Startup(붙임1) 2023년 생애최초 청년창업 지원사업 주관기관 사업계획서...,<1. 작성요령(신청 및 제출 방법)>< ※삭제 후 제출>\r\n<▪ 온라인 신청 ...,https://www.k-startup.go.kr/web/contents/bizpb...
7,K-Startup(붙임2) 2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자...,\r\n\r\n<2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자료>\r\n...,https://www.k-startup.go.kr/web/contents/bizpb...
13,K-Startup(붙임2) 2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료...,\r\n\r\n<2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료>\r\n\...,https://www.k-startup.go.kr/web/contents/bizpb...
14,K-Startup01-2023년 산업맞춤형 혁신 바우처 지원사업 공급기업 모집 공고...,<>\r\n<2023년 산업맞춤형 혁신 바우처 지원사업 공급기업(관) 모집공고>\r...,https://www.k-startup.go.kr/web/contents/bizpb...
20,K-Startup02-2023년 공급기업 Pool 등록 신청양식.hwp,<첨부1><><2023년 산업 맞춤형 혁신 바우처 공급기업(관) Pool 등록 신청...,https://www.k-startup.go.kr/web/contents/bizpb...
26,K-Startup1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp,<제1회 IPO 기술특례상장 지원사업 설명회>\r\n\r\n기술력과 성장...,https://www.k-startup.go.kr/web/contents/bizpb...
33,K-Startup2. 제1회 IPO 기술특례상장 지원사업 설명회 기술사업계획서 양식...,<<><><><><><><><><>\r\n<><><><><><><><><>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
40,K-Startup2023년 공공기술 창업사업화 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 596호\r\n\r\n’23년 공공기술 창업사업화...,https://www.k-startup.go.kr/web/contents/bizpb...
46,K-Startup2023년 생애최초 청년창업 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 595호\r\n\r\n’23년 생애최초 청년창업 ...,https://www.k-startup.go.kr/web/contents/bizpb...


In [13]:
# 인덱스가 꼬였으므로 인덱스를 초기화한다.
hwp_drop.reset_index(inplace=True)

#### 3-2) PDF파일일 경우

In [14]:
!pip install aspose-words

In [15]:
import aspose.words as aw

pdf_texts = []

dir_path # 경로
pdf_file # pdf파일이름 리스트

for i in range(len(pdf_file)) :
    # PDF 파일 로드
    doc = aw.Document(dir_path+'/'+pdf_file[i])
    
    # PDF를 TXT로 저장
    
    if(os.path.exists("pdf-to-text.txt")) :
        os.remove("pdf-to-text.txt")
    
    doc.save("pdf-to-text.txt")

    file = open("./pdf-to-text.txt", "r", encoding='UTF8')
    strings = file.readlines()
    pdf_texts.append(strings)
    file.close()
    
    if(os.path.exists("pdf-to-text.txt")) :
        os.remove("pdf-to-text.txt")
    
pdf_texts

[['\ufeff\n',
  'Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.\n',
  '창업  지원사업  평가위원회  위원  후보단  모집  안내\n',
  '안녕하십니까, 창업진흥원에서 안내드립니다. \n',
  '창업진흥원에서는 창업 지원사업 평가 및 심의·의결 기구 등에 참여할 “창업 지원사업 평가위원회 위원 후보단”을 모집하오니, 관심 있는 분들의 많은 신청 바랍니다. \n',
  '※ 평가관리 시스템 구축 및 평가위원회 위원 후보단 현행화 등에 따라, 기존 K-스타 트업을 통해 등록한 위원도 다시 신청해 주셔야 하며, 향후 시스템에 등록되지 않은 위원은 창업 지원사업 평가 및 심의·의결 기구 등에 참여가 불가합니다.\n',
  '□ 모집개요\n',
  '◦ (추진목적 ) 창업  지원사업  관련  평가  및  심의 ·의결  기구  등에 참여하는 평가위원회 위원 후보단 신규 구축\n',
  '◦ (모집기간 ) 2022년  9월  28일  ~ (후보단  충족  시  마감 )\n',
  '※ 이번 모집을 통해 시스템에 등록되지 않은 평가위원은 ’22년부터 평가, 사업운영 위원회 등에 참여가 불가하오니, 신속히 신청해 주시기 바랍니다. \n',
  '◦ (신청방법 ) K-스타트업  온라인  신청 (www.k-startup.go.kr)\n',
  '※ 접속경로 : K스타트업 로그인(회원가입 후 진행) → 사업 신청관리 → 평가위원 신청\n',
  '※ 세부 신청방법 : (붙임2) 평가위원 신청 가이드 참고\n',
  '◦ (신청자격 ) 아래  자격요건  등에  1개  이상  해당하는  자\n',
  ' \n',
  '구분\n',
  '자격요건\n',
  '예시\n',
  '산업계\n',
  '☞ 박사학위 소지자, 석사학위 소지자 중 해당 분야 5년 이상 경력자, 학사학위 소지자로 해당 분야 7년 이상 경력자 등\n',
  '일반 기업 재직자, 투자자(VC, AC) 

In [16]:
# 잘 들어갔는지 확인
pdf_texts[0]

['\ufeff\n',
 'Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.\n',
 '창업  지원사업  평가위원회  위원  후보단  모집  안내\n',
 '안녕하십니까, 창업진흥원에서 안내드립니다. \n',
 '창업진흥원에서는 창업 지원사업 평가 및 심의·의결 기구 등에 참여할 “창업 지원사업 평가위원회 위원 후보단”을 모집하오니, 관심 있는 분들의 많은 신청 바랍니다. \n',
 '※ 평가관리 시스템 구축 및 평가위원회 위원 후보단 현행화 등에 따라, 기존 K-스타 트업을 통해 등록한 위원도 다시 신청해 주셔야 하며, 향후 시스템에 등록되지 않은 위원은 창업 지원사업 평가 및 심의·의결 기구 등에 참여가 불가합니다.\n',
 '□ 모집개요\n',
 '◦ (추진목적 ) 창업  지원사업  관련  평가  및  심의 ·의결  기구  등에 참여하는 평가위원회 위원 후보단 신규 구축\n',
 '◦ (모집기간 ) 2022년  9월  28일  ~ (후보단  충족  시  마감 )\n',
 '※ 이번 모집을 통해 시스템에 등록되지 않은 평가위원은 ’22년부터 평가, 사업운영 위원회 등에 참여가 불가하오니, 신속히 신청해 주시기 바랍니다. \n',
 '◦ (신청방법 ) K-스타트업  온라인  신청 (www.k-startup.go.kr)\n',
 '※ 접속경로 : K스타트업 로그인(회원가입 후 진행) → 사업 신청관리 → 평가위원 신청\n',
 '※ 세부 신청방법 : (붙임2) 평가위원 신청 가이드 참고\n',
 '◦ (신청자격 ) 아래  자격요건  등에  1개  이상  해당하는  자\n',
 ' \n',
 '구분\n',
 '자격요건\n',
 '예시\n',
 '산업계\n',
 '☞ 박사학위 소지자, 석사학위 소지자 중 해당 분야 5년 이상 경력자, 학사학위 소지자로 해당 분야 7년 이상 경력자 등\n',
 '일반 기업 재직자, 투자자(VC, AC) 등\n',
 '학계\n',
 '☞ 해당

In [17]:
pdf_files = pd.DataFrame(columns=['file_name', 'contents'])

for i in range (len(pdf_file)) :
    pdf_files.loc[len(pdf_files)] = [pdf_file[i], pdf_texts[i]]

pdf_files

,file_name,contents
0,K-Startup(붙임1) 창업사업 평가위원 모집 안내문.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor..."
1,K-Startup(붙임2) 평가위원 신청 가이드.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor..."
2,K-Startup(붙임3) 평가위원 공고 포스터.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor..."
3,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor..."
4,K-Startup[별첨] 2023년 창업중심대학 모집공고 FAQ.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor..."


In [18]:
pdf_inner = pd.merge(pdf_files,crawlingfile,left_on = 'file_name',right_on='file_name',how='inner')
pdf_inner

,file_name,contents,url
0,K-Startup(붙임1) 창업사업 평가위원 모집 안내문.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
1,K-Startup(붙임2) 평가위원 신청 가이드.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
2,K-Startup(붙임3) 평가위원 공고 포스터.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
3,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
4,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
5,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
6,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
7,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
8,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
9,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...


In [23]:
# 중복제거 (에러 발생 >> https://sy-log.tistory.com/60)
# pdf_drop = pdf_inner.drop_duplicates()

TypeError: unhashable type: 'list'

In [27]:
pdf_inner['file_name'] = pdf_inner['file_name'].astype(str)
pdf_drop = pdf_inner.drop_duplicates(['file_name'], keep = 'first')

In [28]:
pdf_drop

,file_name,contents,url
0,K-Startup(붙임1) 창업사업 평가위원 모집 안내문.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
1,K-Startup(붙임2) 평가위원 신청 가이드.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
2,K-Startup(붙임3) 평가위원 공고 포스터.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
3,K-Startup[공고문] 2023년 창업중심대학 모집공고.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...
10,K-Startup[별첨] 2023년 창업중심대학 모집공고 FAQ.pdf,"[﻿\n, Evaluation Only. Created with Aspose.Wor...",https://www.k-startup.go.kr/web/contents/bizpb...


#### 3-3) 이미지 파일일 경우

In [ ]:
from matplotlib import pyplot as plt

from imutils.perspective import four_point_transform
from imutils.contours import sort_contours
import imutils

from easyocr import Reader
import cv2
import requests
import numpy as np
from PIL import ImageFont, ImageDraw, Image

def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [ ]:
dir_path # 경로
img_file # 이미지 파일이름 리스트

img_texts = []

for i in range(len(img_file)) :
    path = dir_path+'/'+img_file[i]
    img_array = np.fromfile(path, np.uint8)
    org_image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    plt_imshow("orignal image", org_image)

    langs = ['ko', 'en']
 
    print("[INFO] OCR'ing input image...")
    reader = Reader(lang_list=langs, gpu=True)
    simple_results = reader.readtext(org_image, detail = 0)
    img_texts.append(simple_results)

img_texts

In [ ]:
#### 3-4) 맞춤법검사API

In [ ]:
from hanspell import spell_checker
import string

# 특수문자 제거
tem_text = []
for i in range(len(img_texts)) :
    tem_text.append(img_texts[i].translate(str.maketrans('', '', string.punctuation)))

spelled_text = spell_checker.check(tem_text)

checked_text = []
for i in range (len(img_texts)) :
    checked_text.append(spelled_text[i].checked)

print(checked_text)

In [ ]:
# 데이터프레임생성
img_files = pd.DataFrame(columns=['file_name', 'contents'])

for i in range (len(img_file)) :
    img_files.loc[len(img_files)] = [img_file[i], checked_text[i]]

img_files

In [ ]:
# url과 병합
img_inner = pd.merge(img_files,crawlingfile,left_on = 'file_name',right_on='file_name',how='inner')
img_inner

In [ ]:
# 중복제거
img_inner['file_name'] = img_inner['file_name'].astype(str)
img_drop = img_inner.drop_duplicates(['file_name'], keep = 'first')

In [ ]:
img_drop

In [ ]:
# 통합하기
hwp_drop
pdf_drop
img_drop



In [29]:
tem_files = hwp_drop.append(pdf_drop)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_9604\2179108122.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tem_files = hwp_drop.append(pdf_drop)


In [30]:
full_files = tem_files.append(img_drop)

,index,file_name,contents,url
0,0.0,K-Startup(붙임1) 2023년 공공기술 창업사업화 지원사업 사업계획서 양식.hwp,<1. 작성요령 (신청 방법 및 기간)>< ※ 해당 페이지 삭제 후 제출>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
1,6.0,K-Startup(붙임1) 2023년 생애최초 청년창업 지원사업 주관기관 사업계획서...,<1. 작성요령(신청 및 제출 방법)>< ※삭제 후 제출>\r\n<▪ 온라인 신청 ...,https://www.k-startup.go.kr/web/contents/bizpb...
2,7.0,K-Startup(붙임2) 2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자...,\r\n\r\n<2023년 공공기술 창업사업화 지원사업 사업계획서 증빙자료>\r\n...,https://www.k-startup.go.kr/web/contents/bizpb...
3,13.0,K-Startup(붙임2) 2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료...,\r\n\r\n<2023년 생애최초 청년창업 지원사업 사업계획서 증빙자료>\r\n\...,https://www.k-startup.go.kr/web/contents/bizpb...
4,14.0,K-Startup01-2023년 산업맞춤형 혁신 바우처 지원사업 공급기업 모집 공고...,<>\r\n<2023년 산업맞춤형 혁신 바우처 지원사업 공급기업(관) 모집공고>\r...,https://www.k-startup.go.kr/web/contents/bizpb...
5,20.0,K-Startup02-2023년 공급기업 Pool 등록 신청양식.hwp,<첨부1><><2023년 산업 맞춤형 혁신 바우처 공급기업(관) Pool 등록 신청...,https://www.k-startup.go.kr/web/contents/bizpb...
6,26.0,K-Startup1. 제1회 IPO 기술특례상장 지원사업 설명회 모집 공고문.hwp,<제1회 IPO 기술특례상장 지원사업 설명회>\r\n\r\n기술력과 성장...,https://www.k-startup.go.kr/web/contents/bizpb...
7,33.0,K-Startup2. 제1회 IPO 기술특례상장 지원사업 설명회 기술사업계획서 양식...,<<><><><><><><><><>\r\n<><><><><><><><><>\r\n<...,https://www.k-startup.go.kr/web/contents/bizpb...
8,40.0,K-Startup2023년 공공기술 창업사업화 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 596호\r\n\r\n’23년 공공기술 창업사업화...,https://www.k-startup.go.kr/web/contents/bizpb...
9,46.0,K-Startup2023년 생애최초 청년창업 지원사업 주관기관 모집공고.hwp,중소벤처기업부 공고 제2022 - 595호\r\n\r\n’23년 생애최초 청년창업 ...,https://www.k-startup.go.kr/web/contents/bizpb...


### DB에 연결 & 저장하기

In [ ]:
!pip install PyMySQL

In [ ]:
import pymysql
import pandas as pd

In [ ]:
columns = ['crl_contents','crl_src','crl_url'] # src는 NaN 이어도 insert 가능, seq와 createdat은 입력 X

In [ ]:
DBinput = pd.DataFrame(columns=columns)

In [ ]:
for i in range(len(full_files)) :
    DBinput.loc[len(DBinput)] = [full_files['contents'][i], full_files['file_name'][i], full_files['url'][i]]

In [ ]:
def insert(sql):
    conn = pymysql.connect(host='project-db-stu.ddns.net', port=3307, user='support24', password='support24', db='support24', charset='utf8')
    with conn:
        with conn.cursor() as cur:
#            cur.execute("set names utf8")    # 없어도 한글 안깨지길래 주석처리함
            cur.execute(sql)
            conn.commit()
            result = cur.fetchall()

In [ ]:
DBinput['crl_src'] = DBinput['crl_src'].fillna('0')
for i in range(len(DBinput)):
    contents = DBinput.iloc[i]['crl_contents']
    src = DBinput.iloc[i]['crl_src']
    url = DBinput.iloc[i]['crl_url']
    sql = '''INSERT INTO tbl_crawling (crl_contents, crl_src, crl_url) VALUES ('%s', '%s', '%s')''' % (contents, src, url)
    insert(sql)